## Q1. Running Elastic 

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?


## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

## Start Solve the problem

```bash
docker run -it \
  --name elasticsearch \
  -p 9200:9200 \
  -p 9300:9300 \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  -e "ES_JAVA_OPTS=-Xms2g -Xmx2g" \
  docker.elastic.co/elasticsearch/elasticsearch:8.17.6

curl localhost:9200

{
  "name" : "b95964286c84",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "VRzy5XfST6-ROrMT_gisZQ",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}
```

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[0]

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* **`index`**
* `put`
* `add`

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch(
    "http://localhost:9200",
    headers={
        "Content-Type": "application/vnd.elasticsearch+json;compatible-with=8",
        "Accept": "application/vnd.elasticsearch+json;compatible-with=8",
    }
)
print(es_client.info())

In [ ]:
index_name = "course-questions"
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

# Create the index
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
# Index the documents
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do execute a command on a Kubernetes pod?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 84.50
* 64.50
* **44.50**
* 24.50

Look at the `_score` field.

In [ ]:
question1 = "How do execute a command on a Kubernetes pod?"

search_query = {
    "query": {
        "multi_match": {
            "query": question1,
            "fields": ["question^4", "text"],  # Boosting question field by 4
            "type": "best_fields"
        }
    }
}

# Execute the search
response = es_client.search(index="course-questions", body=search_query)

# Print the top result with its score
top_hit = response['hits']['hits'][0]
print(f"Top hit score: {top_hit['_score']}")
print(f"Question: {top_hit['_source'].get('question', 'N/A')}")
print(f"Course: {top_hit['_source'].get('course', 'N/A')}")
print(f"Text: {top_hit['_source'].get('text', 'N/A')[:100]}...")

# Print all scores for reference
print("\nAll scores:")
for hit in response['hits']['hits']:
    print(f"{hit['_score']} - {hit['_source'].get('question', 'N/A')[:50]}")

## Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* **How do I copy files from a different folder into docker container’s working directory?**
* How do Lambda container images work?
* How can I annotate a graph?

In [ ]:
question2 = "How do copy a file to a Docker container?"

search_query = {
    "size": 3,  # Return 3 results
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question2,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

# Execute the search
response = es_client.search(index="course-questions", body=search_query)

# Print all three results
for i, hit in enumerate(response['hits']['hits']):
    print(f"Result {i+1}: {hit['_source'].get('question', 'N/A')}")
    print(f"Score: {hit['_score']}")
    print(f"Course: {hit['_source'].get('course', 'N/A')}")
    print("-" * 50)

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 946
* **1446**
* 1946
* 2446

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

# Define the prompt template as specified in the homework
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

# Search query for Q4
search_query = {
    "size": 3,  # Return 3 results
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do copy a file to a Docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

# Execute the search to get records from Q4
response = es_client.search(index="course-questions", body=search_query)

# Build context using the provided template
context = ""
for hit in response['hits']['hits']:
    doc = hit['_source']
    context_entry = context_template.format(
        question=doc.get('question', ''),
        text=doc.get('text', '')
    )
    if context:  # If context is not empty, add two line breaks
        context += "\n\n"
    context += context_entry

# Now build the complete prompt
question = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question=question, context=context)

# Check the length of the prompt
print(f"Prompt length: {len(prompt)}")

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 120
* 220
* **320**
* 420

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [ ]:
import tiktoken

# First, let's make sure we have our prompt from Q5
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

# Execute the search query from Q4
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do copy a file to a Docker container?",
                    "fields": ["question", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index="course-questions", body=search_query)

# Build context using the provided template
context = ""
for hit in response['hits']['hits']:
    doc = hit['_source']
    context_entry = context_template.format(
        question=doc.get('question', ''),
        text=doc.get('text', '')
    )
    if context:  # If context is not empty, add two line breaks
        context += "\n\n"
    context += context_entry

# Build the prompt
question = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question=question, context=context)

# Count tokens using tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
token_count = len(tokens)

print(f"Number of tokens in the prompt: {token_count}")